In [1]:
import librosa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

In [2]:
class Config:
 
    DEBUG_MODE = False
    
    OUTPUT_DIR = './working/'
    DATA_ROOT = './Data'
    FS = 32000
    
    # Mel spectrogram parameters
    N_FFT = 1024
    HOP_LENGTH = 512
    N_MELS = 128
    FMIN = 50
    FMAX = 14000
    
    TARGET_DURATION = 5.0
    TARGET_SHAPE = (256, 256)  
    
    N_MAX = 50 if DEBUG_MODE else None  

config = Config()

In [4]:
df = pd.read_csv(config.DATA_ROOT + '/train.csv')
fabio = df[df.author == 'Fabio A. Sarria-S'].copy()

print(f'We have {len(fabio)} Fabio\'s recordings in total')

We have 26 Fabio's recordings in total


In [20]:
import torch
torch.set_num_threads(1)

from IPython.display import Audio
from pprint import pprint
# download example
torch.hub.download_url_to_file('https://models.silero.ai/vad_models/en.wav', 'en_example.wav')

model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True)

(get_speech_timestamps,
 _, read_audio,
 *_) = utils

sampling_rate = 16000 # also accepts 8000
wav = read_audio('en_example.wav', sampling_rate=sampling_rate)
# get speech timestamps from full audio file
speech_timestamps = get_speech_timestamps(wav, model, sampling_rate=sampling_rate)
pprint(speech_timestamps)

100%|██████████| 1.83M/1.83M [00:01<00:00, 1.79MB/s]
Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to C:\Users\zzannzini/.cache\torch\hub\master.zip


OSError: [WinError 127] 지정된 프로시저를 찾을 수 없습니다